***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# project group 16
# 212240741 323794131
# data retrieval
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ebd0  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage


import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar 23 21:38 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'finaldadabodyupdated' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:      
    if b.name != 'graphframes.sh' and  'docsss' not in b.name and  'postings_gcp' not in b.name and 'postings_new' not in b.name:
        paths.append(full_path+b.name)
        

In [8]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py
import inverted_index_gcp
from inverted_index_gcp import InvertedIndex
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

inverted_index_gcp.py


# Building an inverted index

In [9]:
### functions and variables to be used ###
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){1,24}""", re.UNICODE)
NUM_BUCKETS = 124
time
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, doc_id):
    # applaying stem
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){1,24}""", re.UNICODE)
    ps = nltk.stem.PorterStemmer()
    tokens = [ps.stem(token.group()) for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    word_counts = Counter(tokens)
    posting_list = [(token, (doc_id, tf)) for token, tf in word_counts.items()]
    return posting_list



def reduce_word_counts(unsorted_pl):
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[1])
    return sorted_pl



def calculate_df(postings):
    # Flatten the postings RDD and extract terms
    terms = postings.flatMap(lambda x: [x[0]] * len(x[1]))
    df_counts = terms.map(lambda term: (term, 1)).reduceByKey(lambda a, b: a + b)
    return df_counts



def partition_postings_and_write(postings_filtered):
    posting_grouped = postings_filtered.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    return posting_grouped.map(lambda x: InvertedIndex.write_a_posting_list(x, "postings_new", "finaldadabodyupdated"))


# tokenization
def tokenize_and_steem_word(text):
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){1,24}""", re.UNICODE)
    return [token.group() for token in RE_WORD.finditer(text.lower())]

# all SW
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                "may", "first", "see", "history", "people", "one", "two", 
                "part", "thumb", "including", "second", "following", 
                "many", "however", "would", "became"]
all_stopwords = english_stopwords.union(corpus_stopwords)
# stemmer
ps = nltk.stem.PorterStemmer()

# index init variables update for calculations

inverted = InvertedIndex()
postings_filtered_all = 0
treshold = 50


# title id dict update
parquetFile = spark.read.parquet(*paths)
print('parquetFile')
len_parquet = int(parquetFile.count())
print('creating doc id to title dict...')
doc_id_title = parquetFile.select("id", "title").rdd.map(lambda row: (row.id, row.title))
inverted.idtotitledict.update(doc_id_title.map(lambda x: (x[0], x[1])).collectAsMap())

# doc_len update
print(f'doc_len updating...')
doc_id_len = parquetFile.select("id", "title").rdd.map(lambda row: (row.id, row.title))
length_data = doc_id_len.map(lambda x: (x[0], len([word for word in tokenize_and_steem_word(x[1]) if word not in all_stopwords])))
inverted.doc_len.update(length_data.collectAsMap())

### Index update ###
print(f'Index updating...')
doc_text_pairs = parquetFile.select("title", "id").rdd
# writing to the index
print(f'prapering variables...')
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
print(f'calculating word counts...')
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
print(f'calculating postings_filtered...')
postings_filtered = postings.filter(lambda x: len(x[1])>treshold)
print(f'calculating w2df...')
w2df = calculate_df(postings_filtered)
print(f'updating index df...')
inverted.df.update(w2df.collectAsMap())
print(f'started writing PLs to index...')
_ = partition_postings_and_write(postings_filtered).collect()


 
# collect all posting lists locations into one super-set
print(f'collecting all posting lists locations into one super-set...')

for blob in client.list_blobs(bucket_name, prefix='postings_new'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            if k in inverted.posting_locs.keys():
                inverted.posting_locs[k].extend(v)
            else:
                inverted.posting_locs[k] = v
            
# update index in bucket
print(f'updating index in bucket...')
inverted.write_index('.', 'index')
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst
print('Index has been created !')
    
    
    
    

parquetFile


creating doc id to title dict...


doc_len updating...


Index updating...
prapering variables...
calculating word counts...
calculating postings_filtered...
calculating w2df...
updating index df...


started writing PLs to index...


collecting all posting lists locations into one super-set...
updating index in bucket...
Copying file://index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][212.1 MiB/212.1 MiB]                                                
Operation completed over 1 objects/212.1 MiB.                               

In [34]:
for i in postings_filtered.collect():
    print(i[0])

mad
andrew
hous
holi
high
william
code
actor
support
mile
highway
univers
littl
central
green
want
lee
princess
compani
run
1993
smith
district
dalla
convent
neil
iii
televis
valley
morri
tom
danish
street
georg
offic
jane
servic
england
number
energi
king
lake
lane
frank
germani
jame
battl
bridg
ronald
cricket
dead
toni
bar
black
willi
edward
kelli
2006
gabriel
count
carlo
cup
line
august
1999
manchest
basketbal
point
eye
place
cathol
american
west
robin
washington
ford
medic
head
right
council
episod
law
festiv
guard
surnam
civil
broadcast
rock
alex
irish
opera
protect
presid
1988
elect
scott
countri
conserv
loui
henri
word
1984
sultan
clark
1980
militari
africa
affair
1979
secur
product
food
librari
1976
secondari
leagu
land
walker
metro
fox
limit
cooper
costa
back
free
play
mission
town
infantri
ted
year
wisconsin
suprem
minist
mari
reform
anna
pari
come
challeng
plan
jan
die
singl
pierr
morgan
four
acid
moor
pete
gray
chief
rick
ladi
murder
1960
administr
ann
relat
1968
theodor
ea

[('disambigu',
  [(6899498, 1),
   (6900596, 1),
   (6900735, 1),
   (6901126, 1),
   (6902420, 1),
   (6902516, 1),
   (6903063, 1),
   (6903244, 1),
   (6903790, 1),
   (6904342, 1),
   (6904503, 1),
   (6906611, 1),
   (6908196, 1),
   (6909679, 1),
   (6910288, 1),
   (6910707, 1),
   (6911054, 1),
   (6914506, 1),
   (6914911, 1),
   (6915577, 1),
   (6916358, 1),
   (6916679, 1),
   (6916876, 1),
   (6918965, 1),
   (6919897, 1),
   (6921690, 1),
   (6925335, 1),
   (6926304, 1),
   (6926330, 1),
   (6927272, 1),
   (6927976, 1),
   (6928088, 1),
   (6928105, 1),
   (6928416, 1),
   (6933402, 1),
   (6935611, 1),
   (6936344, 1),
   (6937352, 1),
   (6938701, 1),
   (6939216, 1),
   (6939989, 1),
   (6940726, 1),
   (6941392, 1),
   (6941436, 1),
   (6943042, 1),
   (6943216, 1),
   (6946681, 1),
   (6946902, 1),
   (6947021, 1),
   (6950097, 1),
   (6950144, 1),
   (6951382, 1),
   (6951483, 1),
   (6952256, 1),
   (6952287, 1),
   (6952293, 1),
   (6953152, 1),
   (6953401, 1),